# Project 3
## Joshua Anderson

## General Aspects
1. There will be 10 periods. Players begin with 70 health.
2. Each period, the player harvests some amount of money (this amount is detailed below)
3. After harvesting, the player's health degenerates (this amount is detailed below)
4. After health degeneration, the player must spend money on Health Investments and Life Investments. Money spent on Health Investments increases health, while money spent on Life 5. 5. Investments gives the player Life Enjoyment. Any money not spent carries over into the next period.
6. A player dies if their health ever goes below 0. If a player dies, they receive 0 Life Enjoyment for the remaining periods.
7. The goal is to maximize total Life Enjoyment across all periods.

## Functions
 

Harvesting: 
1. The player earns income by harvesting black dots in a region designated by M x N cells. In our parameters (M = N = 100)
2. The player can select any set of contiguous W columns to harvest. In our parameters (W = 10)
3. When fully healthy, there are T black dots dispersed across M x W cells each period. Each black dot is worth v. In our parameters T = 100 and v = 1
4. The number of rows that can be harvested each period are given by: HarvestRows(H) = M $(1 - \gamma \frac{100 - H}{100})$ where $(\gamma = 1)$
5. The number of rows are reduced by disabling the rows in the upper and lower regions of the selected columns.
6. With these parameters, if health is 50 at the start of the period, the player has only 500 cells in which they can harvest black dots. With 80 health, they have 800 cells.
 

Degeneration:

Each period, the player loses (10 + CurrentPeriod) in health. i.e. 11 health the first period, 12 the second period, up to 20 in the last period.

Health Regeneration:

The equation for the amount of health regained given a certain Health Investment, I, and health after harvesting, H, is given by: 
$$ HealthRegained(I, H) = 100 \left( \frac{e^{k \cdot I}}{e^{k \cdot I} + \frac{100-H}{H}}\right)  - H $$

where (k = 0.01021).  Health cannot exceed 100, and is always rounded down to the nearest integer. 

Life Enjoyment:

The equation for the amount of Life Enjoyment given a certain Life Investment, L, is given by: 

$$ LifeEnjoyment(L, CurrentHealth) = c \left( \frac{CurrentHealth}{100} \right) \left( \frac{L}{L + \alpha} \right) $$

where (c = 464.53) and ($\alpha$ = 32).

CurrentHealth is the health the player has during the investment phase **INCLUDING** the amount regained this period through investments in health.

In [1]:
import numpy as np
import pandas as pd
import math
import random

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

# Constants
periods = 10
population_size = 50

starting_health = 70
min_health = 0

starting_enjoyment = 0

mutate_prob = 0.05
num_children = int(population_size * .75)


In [3]:
### player class ###
class Player:
    def __init__(self, starting_health, starting_enjoyment=None, starting_money=None, starting_I=None, starting_L=None):
        self.health = starting_health

        if starting_enjoyment != None and starting_money != None and starting_I != None and starting_L != None:
            self.enjoyment = starting_enjoyment
            self.money = starting_money
            self.I_prop = starting_I
            self.L_prop = starting_L
        else:
            self.enjoyment = 0
            self.money = 0
            self.set_investments()
    
    
    def invest(self):
        L = self.L_prop * self.money
        I = self.I_prop * self.money
        self.money -= L
        self.money -= I

        return I, L

    def set_investments(self):
        self.L_prop = random.random()
        self.I_prop = 1 - self.L_prop

In [4]:
### simulation functions ###

# Harvesting
def harvest(players, M = 100, N = 100, W = 10, T = 100, v = 1, gamma = 1):
    dots_per_cell = T / (M * W)

    for player in players:
        num_player_rows = M * (1 - (gamma * ((100 - player.health) / 100)))
        player.money = num_player_rows * W * dots_per_cell * v
        
# Degeneragtion
def degenerate(curr_period, P = 10):
    return P + curr_period
    

# Health Regeneration
def regenerate(I, health, k = 0.01021, max_health = 100):
        e_kI = math.exp(k*I)
        H_frac = (max_health - health) / health
        
        health_regen = math.floor(max_health * (e_kI / (e_kI + H_frac)) - health)

        if health_regen > 0:
            if health_regen > max_health:
                return max_health
            else:
                return health_regen
        
        return 0

# Life Enjoyment - Fitness Function
def enjoyment(L, health, c = 464.53, alpha = 32):
        return c * (health / 100) * (L / (L + alpha))

# make investments
def invest_pop(players):
    for player in players:
        if player.health > min_health:
            I, L = player.invest()
            player.health += regenerate(I, player.health)
            player.enjoyment += enjoyment(L, player.health)

# helper function to print population
def print_players(players):
    print("    H  -   LE   -   I  -   L")
    print(np.array([(player.health, player.enjoyment, player.I_prop, player.L_prop) for player in players]))

In [41]:
### GA functions ###

# generate population
def generate_players(starting_health, population_size):
    return [Player(starting_health) for _ in range(population_size)]

def mutate_investments(player, prob):
    if random.random() < prob:
        mutate_prop = random.uniform(0, 0.2)

        if random.random() < 0.5:
            if player.I_prop + mutate_prop > 1:
                player.I_prop = 1
            else:
                player.I_prop += mutate_prop
            if player.L_prop - mutate_prob < 0:
                player.L_prop = 0
            else:
                player.L_prop -= mutate_prop
        
        else:
            if player.L_prop + mutate_prop > 1:
                player.L_prop = 1
            else:
                player.L_prop += mutate_prop
            if player.I_prop - mutate_prob < 0:
                player.I_prop = 0
            else:
                player.I_prop -= mutate_prop
    return player

# generate children
def generate_children(players, num_children, mutate_prob):
    children = []

    for _ in range(num_children):
        parent_1, parent_2 = random.sample(players, 2)
        I_prop = 0
        L_prop = 0

        if random.random() < 0.5:
            L_prop = parent_2.L_prop
            if parent_1.I_prop + parent_2.L_prop <= 1:
                I_prop = parent_1.I_prop
                
            child = Player(parent_1.health, parent_2.enjoyment, parent_1.money, I_prop, L_prop)
            children.append(mutate_investments(child, mutate_prob))
        else:
            L_prop = parent_1.L_prop
            if parent_2.I_prop + parent_1.L_prop <= 1:
                I_prop = parent_2.I_prop
            child = Player(parent_2.health, parent_1.enjoyment, parent_2.money, I_prop, L_prop)
            children.append(mutate_investments(child, mutate_prob))
            
    return children

# fitness function
def investment_fitness(player, curr_period):
    score = 0

    if min_health < player.health - degenerate(curr_period) and player.I_prop < player.L_prop:
        score -= 250
    
    score += player.enjoyment

    return score

# tournament survival
def reduce_players(players, population_size, curr_period):
    new_players = []

    for _ in range(population_size):
        player_1, player_2 = random.sample(players, 2)

        if investment_fitness(player_1, curr_period) > investment_fitness(player_2, curr_period):
            new_players.append(player_1)
        else:
            new_players.append(player_2)
    return new_players

# iterate the GA
def ga_iteration(players, curr_period, mutate_prob, population_size):
    # print(f"Period {curr_period+1}:")
    harvest(players)
    
    for player in players:
        if min_health < (player.health - degenerate(curr_period)):
            player.health -= degenerate(curr_period)
        else:
            player.health = 0

    invest_pop(players)

    children = generate_children(players, num_children, mutate_prob)
    new_players = reduce_players(players + children, population_size, curr_period)

    return new_players

In [48]:
players = generate_players(starting_health, population_size)

curr_pop = players
for curr_period in range(10):
    curr_pop = ga_iteration(curr_pop, curr_period, mutate_prob, population_size)

print_players(curr_pop)

    H  -   LE   -   I  -   L
[[  0.   727.6    0.     0.61]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.37   0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.52   0.48]
 [  0.   727.6    0.37   0.63]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.52   0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   682.34   0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.52   0.48]
 [  0.   727.6    0.52   0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   682.34   0.     0.48]
 [  0.   727.6    0.     0.48]
 [  0.   727.6    0.52   0.48]
 [  0.   727.6    0.     0.61]
 [  0.   727.6    0.     0.61]
 [  0.   7